# 介绍

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/renweibo/stable-diffusion-webui-colab/blob/master/chillout_mix_webui_colab.ipynb)

In [ ]:
#@title 基本配置 {display-mode: "form"}

#@markdown Forms support many types of fields.
root_path = '/content'  #@param {type: "string"}
sd_path = '/content/stable-diffusion-webui'  #@param {type: "string"}
base_install_flag = False #@param {type:"boolean"}
ser_start_flag = False #@param {type:"boolean"}
#@markdown ---

In [ ]:
# 安装运行

In [ ]:
#@title 公共函数 {display-mode: "form"}
!pip install -q -U sh
!apt -y update -qq
!apt install -qq aria2
import sh
from pathlib import Path
def aria2c(url, path):
    file = Path(url).name
    sh.aria2c(f"--console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {sd_path}/{path} -o {file}".split())
    !ls -lh -d "$sd_path/$path/$file"
def git_clone(url, path:str=None, branch=None):
    file = Path(url).name
    if path is None or path.strip()==".":
        p = Path(root_path, file)
    else:
        p = Path(sd_path, path)
    if p.exists():
        if p.is_dir():
            display(f"{p} exists")
        else:
            display(f"{p} is not directory")
    branch_str = f"-b {branch}" if branch is not None else ""
    if path is None or path.strip()==".":
        sh.git(f"clone {branch_str} {url}".split(), _cwd=str(p.parent)) 
    else:
        sh.git(f"clone {branch_str} {url} {str(p)}".split()) 
    !du -csh "$p"
# git_clone("https://github.com/camenduru/stable-diffusion-webui", ".", "v2.1")
# git_clone("https://github.com/camenduru/stable-diffusion-webui", branch="v2.1")
# !rm -rf /content/stable-diffusion-webui/embeddings/negative
# git_clone("https://huggingface.co/embed/negative", "embeddings/negative")
    

In [ ]:
#@title ## 安装基础软件 {display-mode: "form"}
%cd "$root_path"
%env TF_CPP_MIN_LOG_LEVEL=1

if base_install_flag:
    !wget -q http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
    !wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
    !wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
    !wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
    !apt install -qq libunwind8-dev
    !dpkg -i *.deb
    %env LD_PRELOAD=libtcmalloc.so
    !rm *.deb

    !pip install -q -U torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116
    !pip install -q -U xformers==0.0.16 triton==2.0.0

In [ ]:
#@title 下载Stable Diffusion Web UI {display-mode: "form"}
!rm -rf "$sd_path"
%cd "$root_path"
git_clone("https://github.com/camenduru/stable-diffusion-webui", branch="v2.1")
%cd "$sd_path"
!git reset --hard
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' "$sd_path"/launch.py
!sed -i -e 's/fastapi==0.90.1/fastapi==0.89.1/g' "$sd_path"/requirements_versions.txt

aria2c("https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth", "models/ESRGAN")

!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O "$sd_path"/scripts/run_n_times.py

git_clone("https://huggingface.co/embed/negative", "embeddings/negative")
git_clone("https://huggingface.co/embed/lora", "models/Lora/positive")

git_clone("https://github.com/deforum-art/deforum-for-automatic1111-webui", "extensions/deforum-for-automatic1111-webui")
!mkdir -p "$sd_path"/extensions/deforum-for-automatic1111-webui/models
!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' "$sd_path"/modules/shared.py

git_clone("https://github.com/camenduru/stable-diffusion-webui-images-browser", "extensions/stable-diffusion-webui-images-browser")
git_clone("https://github.com/camenduru/stable-diffusion-webui-huggingface", "extensions/stable-diffusion-webui-huggingface")
git_clone("https://github.com/camenduru/sd-civitai-browser", "extensions/sd-civitai-browser")
git_clone("https://github.com/kohya-ss/sd-webui-additional-networks", "extensions/sd-webui-additional-networks")
git_clone("https://github.com/Mikubill/sd-webui-controlnet", "extensions/sd-webui-controlnet")
git_clone("https://github.com/camenduru/openpose-editor", "extensions/openpose-editor")
git_clone("https://github.com/jexom/sd-webui-depth-lib", "extensions/sd-webui-depth-lib")
git_clone("https://github.com/hnmr293/posex", "extensions/posex")
git_clone("https://github.com/camenduru/sd-webui-tunnels", "extensions/sd-webui-tunnels")
git_clone("https://github.com/etherealxx/batchlinks-webui", "extensions/batchlinks-webui")
git_clone("https://github.com/camenduru/stable-diffusion-webui-catppuccin", "extensions/stable-diffusion-webui-catppuccin")
git_clone("https://github.com/KohakuBlueleaf/a1111-sd-webui-locon", "extensions/a1111-sd-webui-locon")
git_clone("https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg", "extensions/stable-diffusion-webui-rembg")
git_clone("https://github.com/ashen-sensored/stable-diffusion-webui-two-shot", "extensions/stable-diffusion-webui-two-shot")
git_clone("https://github.com/camenduru/sd_webui_stealth_pnginfo", "extensions/sd_webui_stealth_pnginfo")


In [ ]:
#@title 下载公共数据 {display-mode: "form"}
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_canny-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_depth-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_hed-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_mlsd-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_normal-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_openpose-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_scribble-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_seg-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth", "extensions/sd-webui-controlnet/annotator/openpose")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth", "extensions/sd-webui-controlnet/annotator/openpose")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt", "extensions/sd-webui-controlnet/annotator/midas")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth", "extensions/sd-webui-controlnet/annotator/mlsd")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth", "extensions/sd-webui-controlnet/annotator/mlsd")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth", "extensions/sd-webui-controlnet/annotator/hed")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth", "extensions/sd-webui-controlnet/annotator/uniformer")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_style_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_sketch_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_seg_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_openpose_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_keypose_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_depth_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_color_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_canny_sd14v1.pth", "extensions/sd-webui-controlnet/models")

In [ ]:
#@title 下载主模型 {display-mode: "form"}
# chillout mix
aria2c("https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors", "models/Stable-diffusion")

# waifu_diffusion_v1_5_webui_colab
aria2c("https://huggingface.co/ckpt/wd-1-5-beta/resolve/main/wd15-beta1-fp32.safetensors", "models/Stable-diffusion")
aria2c("https://huggingface.co/ckpt/wd-1-5-beta/raw/main/wd15-beta1-fp32.yaml", "models/Stable-diffusion")
aria2c("https://huggingface.co/ckpt/waifu-diffusion-v1-4/resolve/main/kl-f8-anime2.ckpt", "models/Stable-diffusion")

# RPG-v4
aria2c("https://huggingface.co/Anashel/rpg/resolve/main/RPG-V4-Model-Download/RPG-v4.safetensors", "models/Stable-diffusion")
aria2c("https://huggingface.co/Anashel/rpg/raw/main/RPG-V4-Model-Download/RPG-v4.yaml", "models/Stable-diffusion")
aria2c("https://huggingface.co/Anashel/rpg/resolve/main/RPG-VAE-Current-Download/vae-ft-mse-840000-ema-pruned.ckpt", "models/Stable-diffusion")

# anything v3.0
#aria2c("https://huggingface.co/ckpt/anything-v3.0/resolve/main/Anything-V3.0-pruned.ckpt", "models/Stable-diffusion")
#aria2c("https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt", "models/Stable-diffusion")
# mv "$sd_path"/models/Stable-diffusion/vae-ft-mse-840000-ema-pruned.ckpt "$sd_path"/models/Stable-diffusion/Anything-V3.0-pruned.vae.pt

# anything v4.5
aria2c("https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.5-pruned.safetensors", "models/Stable-diffusion")
aria2c("https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.0.vae.pt", "models/Stable-diffusion")
!mv "$sd_path"/models/Stable-diffusion/anything-v4.0.vae.pt "$sd_path"/models/Stable-diffusion/anything-v4.5.vae.pt
# aria2c("https://huggingface.co/ckpt/anything-v5.0/blob/main/AnythingV5V3_v5PrtRE.safetensors", "models/Stable-diffusion")

# dreamlike-photoreal-2.0
aria2c("https://huggingface.co/ckpt/dreamlike-photoreal-2.0/resolve/main/dreamlike-photoreal-2.0.safetensors", "models/Stable-diffusion")

# openjourney-v2
aria2c("https://huggingface.co/ckpt/openjourney-v2/resolve/main/openjourney-v2.ckpt", "models/Stable-diffusion")

# dreamlike-diffusion-1.0
aria2c("https://huggingface.co/ckpt/dreamlike-diffusion-1.0/resolve/main/dreamlike-diffusion-1.0.ckpt", "models/Stable-diffusion")

# stable-diffusion-2.1
aria2c("https://huggingface.co/ckpt/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt", "models/Stable-diffusion")
aria2c("https://huggingface.co/ckpt/stable-diffusion-2-1/raw/main/v2-inference-v.yaml", "models/Stable-diffusion")
!mv "$sd_path"/models/Stable-diffusion/v2-inference-v.yaml "$sd_path"/models/Stable-diffusion/v2-1_768-ema-pruned.yaml

aria2c("https://huggingface.co/joachimsallstrom/Double-Exposure-Embedding/resolve/main/dblx.pt", "embeddings")
aria2c("https://huggingface.co/Conflictx/AnimeScreencap/resolve/main/AnimeScreenCap.pt", "embeddings")

# midjourney-v4-diffusion
aria2c("https://huggingface.co/ckpt/midjourney-v4-diffusion/resolve/main/mdjrny-v4.ckpt, "models/Stable-diffusion")

# stable-diffusion-1.5
aria2c("https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt, "models/Stable-diffusion")

# Counterfeit V2.0 fp16
aria2c("https://huggingface.co/ckpt/Counterfeit-V2.0/blob/main/Counterfeit-V2.0fp16.safetensors, "models/Stable-diffusion")

In [ ]:
#@title 运行服务 {display-mode: "form"}
if ser_start_flag:
    %cd "$sd_path"
    !python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple